In [1]:
import os
import numpy as np
import librosa
import soundfile as sf
from scipy.signal import butter, sosfilt

# directoare output
dirs = ['audio_clean', 'audio_white_noise', 'audio_low_pass', 'audio_high_pass', 'audio_mid_pass']
for d in dirs:
    os.makedirs(d, exist_ok=True)

# functie Butterworth, practic cu ea taiem frecvente
def butter_filter(y, sr, cutoff, btype='low', order=4):
    nyq = 0.5 * sr
    if btype == 'bandstop':
        normal_cutoff = [cutoff[0] / nyq, cutoff[1] / nyq]
    else:
        normal_cutoff = cutoff / nyq
    sos = butter(order, normal_cutoff, btype=btype, analog=False, output='sos')
    return sosfilt(sos, y)

# pentru adaugat white noise aleator cu signal-noise-ratio 12
def add_white_noise(y, snr=12):
    signal_power = np.mean(y**2)
    noise_power = signal_power / (10**(snr / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), len(y))
    return y + noise

audio_dir = 'audio/'
for filename in os.listdir(audio_dir):
    if filename.endswith('.wav'):
        file_path = os.path.join(audio_dir, filename)
        
        # nu trebuie resample teoretic caci sunt deja la 44100 kHz
        y, sr = librosa.load(file_path, sr=44100)
        y = y / np.max(np.abs(y)) if np.max(np.abs(y)) != 0 else y
        
        clean_path = os.path.join('audio_clean', filename)
        sf.write(clean_path, y, sr)

        # white noise
        y_noisy = add_white_noise(y)
        noisy_path = os.path.join('audio_white_noise', filename)
        sf.write(noisy_path, y_noisy, sr)
        
        # low pass
        y_low = butter_filter(y, sr, 1500, 'low')
        low_path = os.path.join('audio_low_pass', filename)
        sf.write(low_path, y_low, sr)
        
        # high pass
        y_high = butter_filter(y, sr, 2500, 'high')
        high_path = os.path.join('audio_high_pass', filename)
        sf.write(high_path, y_high, sr)
        
        # mid pass
        y_mid = butter_filter(y, sr, [2500, 4500], 'bandstop')
        mid_path = os.path.join('audio_mid_pass', filename)
        sf.write(mid_path, y_mid, sr)

print("Procesare completa")

Procesare completa
